In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

     |████████████████████████████████| 911kB 2.7MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
def help_scroll(driver):
  #функция чтоб пролистать вниз списка

  help = driver.find_elements_by_class_name('event__match')
  driver.execute_script("arguments[0].scrollIntoView();", help[-3])
  print(len(help))
def check_more_matches(driver):
  #проверка, можно ли еще прогружать список матчей

  elements = driver.find_elements_by_class_name('event__more')
  if(len(elements) > 0):
    return True
  else:
    return False
def load_more_matches(driver):
  #прогружаем еще матчи

  try:
    element = driver.find_elements_by_class_name('event__more')[0]
    help_scroll(driver)
    element.click()
  except MoveTargetOutOfBoundsException:
    print('fuck')

In [ ]:
def get_id_matches_in_season(season):
  # получить id всех матчей в сезоне

  ids = []

  driver = webdriver.Chrome(options=chrome_options)
  driver.get(season)
  
  while True:
    if(check_more_matches(driver)):
      load_more_matches(driver)
      time.sleep(2)
    else:
      break
  
  html = driver.page_source
  soap = BeautifulSoup(html)

  for i in soap.find_all('div', class_='event__match'):
    ids.append(i['id'].split('_')[2])
  
  driver.close()

  return ids

In [ ]:
def stats_game_in_series(driver):
  #получить статистику матча
  
  soap = BeautifulSoup(driver.page_source)
  stats = soap.findAll('div', class_='statCategory___33LOZ_7')
  
  data = []
  index = []
  
  for i in stats:
    for ind, child in enumerate(i):
      if(ind in [0,2]):
        data.append(child.text.replace('%', ''))
      else:
        index.append('Дома_'+child.text)
        index.append('Гости_'+child.text)
  
  return pd.Series(data=data, index=index)

In [ ]:
def same_sign(a, b):
  return (a / b) >= 0

def calculate_scores_by_qtr(driver):
  #получить "ход матча"
  
  res = []
  
  elements = driver.find_elements_by_class_name('subTabs__tab')
  for i in elements:
    try:
      i.click() 
      sleep(0.5)
    except:
      None
    
    soap = BeautifulSoup(driver.page_source)
    
    for rows in soap.find_all('div', class_='matchHistory___3SdQ7EQ'):
      for ind, score in enumerate(rows):
        if(ind == 1):
          res.append(score.text)

  return res

def calculate_metric_pbp(pbp):
  # по "ход матча" посчитать 'Рав-во_счета', 'Смена_лид-ва', 'Дома_макс_лид-во', 'Гости_макс_лид-во'

  equals = 0
  change_lead = 0
  max_lead_home = pbp.max()
  max_lead_away = abs(pbp.min())
  
  for i in range(len(pbp)-1):
    if (pbp[i] == 0):
      equals +=1
      if (~same_sign(pbp[i+1], pbp[i-1])):
        change_lead +=1
    
    elif (~same_sign(pbp[i+1], pbp[i])):
      change_lead += 1 
  
  return pd.Series(data=[equals, change_lead, max_lead_home, max_lead_away],
                   index=['Рав-во_счета', 'Смена_лид-ва', 'Дома_макс_лид-во', 'Гости_макс_лид-во'])

def calculate_run(dif):
  #по "ход матча" посчитать 'Дома_макс_рывок', 'Гости_макс_рывок', 'Дома_кол-во_рывков', 'Гости_кол-во_рывков'

  for i in range(1, len(dif)):
    if(same_sign(dif[i], dif[i-1])):
      dif[i] += dif[i-1]
  
  max_home = dif.max()
  max_away = abs(dif.min())
  q_run_home = len(dif[dif > 5])
  q_run_away = len(dif[dif < -5])
  
  return pd.Series(data=[max_home, max_away, q_run_home, q_run_away],
                   index=['Дома_макс_рывок', 'Гости_макс_рывок', 'Дома_кол-во_рывков', 'Гости_кол-во_рывков'])

def scores_game(driver):
  #объеденить метрики из "ход матча"

  check = calculate_scores_by_qtr(driver)
  
  home = []
  away = []
  
  for i in check:
    home_point, away_point = i.split('-')
    home.append(int(home_point))
    away.append(int(away_point))
  
  delta_point = np.array(home) - np.array(away)

  scores_metric = calculate_metric_pbp(delta_point)

  dif = []
  for i in range(1, len(delta_point)):
    dif.append(delta_point[i] - delta_point[i-1])
  dif = np.array(dif)

  run_metric = calculate_run(dif)

  return pd.concat([scores_metric, run_metric])

In [ ]:
def check_href(driver, href):
  #проверка существует ли элемент с данной ссылкой, и клик на него, если да

  elem = driver.find_elements_by_xpath("//a[@href='" + href + "']")
  if (len(elem)):
    try:
      elem[0].click()
      time.sleep(1)
    except:
      driver.find_element_by_id('onetrust-accept-btn-handler').click()
      elem[0].click()
      time.sleep(1)  
    return True   

In [ ]:
#функции для подсчета коэффициентов на победу, тотала с примерно равными коэффициентами, тотала 1ой четверти
#которые дают букмекеры(без указания букмекера)
def odds_over_under(driver, name_event, bound):

  over = []
  under = []
  total = []
  
  soap = BeautifulSoup(driver.page_source)
  
  for first in soap.find_all('div', class_='row___1rtP1QI'):#rows___1BdItrT'):
  #for second in first:
    for ind, odds in enumerate(first):#second):
      if(ind == 1):
        if (float (odds.text) < bound):
          break
        total.append(odds.text)
      if(ind == 2):
        over.append(float(odds.text))
      if(ind == 3):
        under.append(float(odds.text))
  try:
    ind = np.argmin(abs(np.array(over) - np.array(under)))
    return pd.Series([total[ind], over[ind], under[ind]], index=[name_event, name_event+'_больше',
                                                                 name_event+'_меньше'])
  except:
    return pd.Series(['empty']*3, index=[name_event, name_event+'_больше',
                                                                 name_event+'_меньше'])
    

def odds_match(driver):
  series = []
  
  home = []
  away = []
  
  soap = BeautifulSoup(driver.page_source)
  for first in soap.find_all('div', class_='rows___1BdItrT'):
    for second in first:
      for ind, odds in enumerate(second):
        if(ind == 1):
          home.append((odds.text))
        if(ind == 2):
          away.append((odds.text))
  
  series.append(pd.Series(data=[home[0], away[0]], index=['Дома_кэф_победа', 'Гости_кэф_победа']))
  
  if (check_href(driver, '#odds-comparison/over-under')):
    series.append(odds_over_under(driver, 'Тотал_прогноз_матча', 150))

    if (check_href(driver, '#odds-comparison/over-under/1st-qrt')):
      series.append(odds_over_under(driver, 'Тотал_1ч_прогноз', 20))
  
  return pd.concat(series)

In [ ]:
def get_one_match(id):
  #получить данные одного матча по его id

  series = []

  url = 'https://www.flashscore.ru/match/' + id + '/#match-summary/match-summary'
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(url)
  time.sleep(0.5)

  soap = BeautifulSoup(driver.page_source)

  description = [soap.find_all('span', class_='country___24Qe-aj')[0].select('a')[0].text]
  info = soap.find_all('div', class_='info___2fPCmvW')
  if len(info):
    info = [info[0].text]
  else:
    info = ['empty']
  date_time_match = soap.find_all('div', class_='startTime___2oy0czV')[0].text.split()

  away_score = soap.find_all('div', class_='away___2Q2jzQu')[1:]
  away_score = [i.text for i in away_score]

  home_score = soap.find_all('div', class_='home___2hIlHif')[1:]
  home_score = [i.text for i in home_score]

  columns = ['Описание', 'Дата', 'Время', 'Дома_название', 'Дома_счет_матча', 'Дома_счет_1ч',
            'Дома_счет_2ч', 'Дома_счет_3ч', 'Дома_счет_4ч', 'Дома_счет_от', 'Гости_название',
            'Гости_счет_матча', 'Гости_счет_1ч', 'Гости_счет_2ч', 'Гости_счет_3ч', 'Гости_счет_4ч',
            'Гости_счет_от', 'Информация']
  series.append(pd.Series(data=description+date_time_match+home_score+away_score+info, index=columns))

  if (check_href(driver, '#match-summary/match-statistics')):
    series.append(stats_game_in_series(driver))

  if (check_href(driver, '#match-summary/point-by-point')):
    series.append(scores_game(driver))

  if (check_href(driver, '#odds-comparison')):
    series.append(odds_match(driver))
 
  driver.close()
  return pd.concat(series)

In [ ]:
def get_all_games(season):
  #получить dataframe из всех id матчей определенного сезона 

  error = []
  series = []
  for i in log_progress(season):
    try:
      series.append(get_one_match(i))
    except:
      error.append(i)
      print(error)
  return pd.concat(series, axis=1).T, error

In [ ]:
response = requests.get('https://www.flashscore.ru/basketball/usa/nba/archive/')
response

<Response [200]>

In [ ]:
soup = BeautifulSoup(response.text)

In [ ]:
seasons = []
#получим названия сезонов
season_ref = []
#получим ссылки на эти сезоны

for i in soup.find_all('div', class_='leagueTable__seasonName'):
  spl = i.text.split()
  
  if((spl[0]) == 'НБА'):
    seasons.append(spl[1])
    season_ref.append('https://www.flashscore.ru' + i.a['href'] + 'results/')

In [ ]:
last_season = season_ref[0]
last_season
#крайний сезон

'https://www.flashscore.ru/basketball/usa/nba/results/'

In [ ]:
'''def get_games_by_ref(ref, name):
  id_in_season = get_id_matches_in_season(ref)
  df_season, error_season = get_all_games(id_in_season[:5])
  df_download(df_season, name+'.csv')'''

"def get_games_by_ref(ref, name):\n  id_in_season = get_id_matches_in_season(ref)\n  df_season, error_season = get_all_games(id_in_season[:5])\n  df_download(df_season, name+'.csv')"

In [ ]:
#get_games_by_ref(season_ref[0], seasons[0])

In [ ]:

s15_16 = season_ref[5]
id_s15_16 = get_id_matches_in_season(s15_16)


114
203
302
405
512
603
708
808
900
1000
1104
1206
1300
1402


In [ ]:
df_s15_16, error_s15_16 = get_all_games(id_s15_16[:300])

['2HtRpIuj']
['2HtRpIuj', 'Kx7PtCrm']
['2HtRpIuj', 'Kx7PtCrm', 'QBi8nAJt']


In [ ]:
len(s)

2

In [ ]:
s = []
for i in error_s15_16:
  er = get_one_match(i)
  s.append(er)

In [ ]:
pd.concat([df_s15_16, pd.concat(s, axis=1).T]).to_csv('15_16_300.csv', index=False, encoding='utf-8')

In [ ]:
df_s16_17.to_csv('16_17_500_1000.csv', index=False, encoding='utf-8')